<a href="https://colab.research.google.com/github/rahiakela/python-for-data-analysis/blob/7-data-cleaning-and-preparation/2_data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Transformation

Filtering, cleaning, and other transformations are another class of important operations.

## Removing Duplicates

In [1]:
import pandas as pd
import numpy as np


# Duplicate rows may be found in a DataFrame for any number of reasons.
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1, 1, 2, 3, 3, 4, 4]
})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [2]:
# method duplicated returns a boolean Series indicating whether each row is a duplicate or not
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [3]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. 

Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column.

In [4]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [5]:
data.drop_duplicates(['k1'])  # drop duplicate based on first column

,k1,k2,v1
0,one,1,0
1,two,1,1


In [6]:
data.drop_duplicates(['k2'])  # drop duplicate based on second column

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


duplicated and drop_duplicates by default keep the first observed value combination. Passing keep='last' will return the last one.

In [7]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

Consider the following hypothetical data collected about various kinds of meat:

In [8]:
data = pd.DataFrame({
    'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. 

Let’s write down a mapping of each distinct meat type to the kind of animal:

In [0]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. 

Thus, we need to convert each value to lowercase using the str.lower Series method:

In [10]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [12]:
# do it in one-shot
data['animal'] = data['food'].str.lower().map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work.

In [13]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

## Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. 

As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so.

In [14]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True).

In [15]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value.

In [16]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes.

In [17]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict.

In [18]:
data.replace({-999:np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure.

In [19]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), 
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [20]:
# transform index to upper-case and trim to length 4
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [21]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


You can assign to index, modifying the DataFrame in-place.

In [22]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is rename.

In [23]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels.

In [24]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:

In [25]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis.

Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [0]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use cut, a function in pandas:

In [27]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by pandas.cut. You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute:

In [28]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [29]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [30]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

You can also pass your own bin names by passing a list or array to the labels option:

In [31]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

## Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.

In [32]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.110948,-0.016802,0.023274,-0.033717
std,1.014400,0.986352,0.987271,1.013709
min,-3.415422,-3.572513,-3.178858,-3.733717
25%,-0.820411,-0.683111,-0.651780,-0.724703
50%,-0.107894,-0.043464,0.034761,-0.019291
75%,0.557663,0.634734,0.676153,0.664531
max,3.477196,3.544656,3.507651,2.983374


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [33]:
col = data[2]
col[np.abs(col) > 3]

827   -3.178858
971    3.507651
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame:

In [34]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
124,0.557378,0.283442,0.181691,-3.733717
264,-0.412396,-3.572513,-0.557802,1.650128
296,-0.970042,0.604672,-1.315480,-3.579570
315,-1.449584,-3.151990,1.536468,-0.967891
406,-3.415422,-0.590509,-0.407877,-1.930785
430,3.477196,-0.700587,-0.694773,-0.540619
701,-0.533086,3.544656,-0.816556,0.002044
713,1.317915,-3.108440,1.142713,1.736130
724,3.396487,-0.802667,-0.396434,0.795244
827,0.966356,-1.375780,-3.178858,-0.719660


Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [35]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.111406,-0.016514,0.022945,-0.032404
std,1.010250,0.981817,0.985041,1.009420
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.820411,-0.683111,-0.651780,-0.724703
50%,-0.107894,-0.043464,0.034761,-0.019291
75%,0.557663,0.634734,0.676153,0.664531
max,3.000000,3.000000,3.000000,2.983374


The statement np.sign(data) produces 1 and –1 values based on whether the values in data are positive or negative:

In [36]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,-1.0,1.0,1.0,-1.0


## Permutation and Random Sampling